# Calibration From Scratch
This notebook shows the process of calibrating the extrinsic parameters between the Vicon marker coordinate frame installed on the end-effector of a robot with respect to the end-effector frame.

### Vicon Sensor

In [ ]:
from SimpleHandEye.interfaces.ros2 import ROS2ExecutorManager, ROS2CameraReader, ROS2TFInterface
import rclpy
rclpy.init()    
executor_manager = ROS2ExecutorManager()
vicon_sensor = ROS2TFInterface('vicon/World', 'vicon/WHITE_FR3_EF/WHITE_FR3_EF', node_name='vicon_sensor_node')
executor_manager.add_node(vicon_sensor)
executor_manager.start()

In [ ]:
world_T_marker = vicon_sensor.get_pose()
world_T_marker

### FR3 Forward Kinematic

In [ ]:
 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

In [ ]:
from FR3Py.robot.model import PinocchioModel
model = PinocchioModel()

In [ ]:
import numpy as np
def getFK(robot, model):
    state = robot.getJointStates()
    q , dq = state['q'], state['dq']
    info = model.getInfo(q, dq)
    R_ee = info['R_EE']
    t_ee = info['P_EE']
    T = np.hstack([R_ee, t_ee.reshape(3,1)])
    T = np.vstack([T, np.array([0, 0, 0, 1])])
    return T

base_T_ef= getFK(robot, model)
base_T_ef

## Instantiate the Solver

In [ ]:
from SimpleHandEye.solvers import OpenCVSolver
solver = OpenCVSolver()

## Collect The Calibration Dataset

In [ ]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from pprint import pprint
from IPython.display import clear_output
np.set_printoptions(suppress=True, precision=3)

# The dataset
A_list = []
B_list = []
def on_sample_clicked(b):
    A = getFK(robot, model)
    B = vicon_sensor.get_pose()
    print("A=")
    pprint(A)
    print("B=")
    pprint(B)
    A_list.append(A)
    B_list.append(B)
    print("*************")

def on_compute_clicked(b):
    try:
        X,Y = solver.solve(A_list, B_list)
        clear_output(wait=True)
        print("X=")
        pprint(X)
        print("Y=")
        pprint(Y)
    except:
        print("Bad dataset, please record again")
        A_list.clear()
        B_list.clear()
        

sample_button = widgets.Button(description="Sample")
compute_button = widgets.Button(description="Compute")

sample_button.on_click(on_sample_clicked)
compute_button.on_click(on_compute_clicked)
display(sample_button)
display(compute_button)

In [ ]:
X, Y = solver.solve(A_list, B_list)

In [ ]:
ef_T_marker = X
Base_T_viconW = Y

In [ ]:
for i in range(len(A_list)):
    print((A_list[i]@X@np.linalg.inv(Y@B_list[i]))[:3,-1])

In [ ]:
import pickle
# camera_name = 'side_left_cam'
dataset_name = f'fr3-vicon-dataset.pkl'
with open(dataset_name, 'wb') as f:
    data = {
        'A_list': A_list,
        'B_list': B_list,
        'X': X,
        'Y': Y,
        'ef_T_marker':ef_T_marker,
        'Base_T_viconW':Base_T_viconW
    }
    pickle.dump(data, f)

# Get base_T_world From Previously Estimated ef_T_marker

In [ ]:
import pickle
with open(f'fr3-vicon-dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    A_list = data['A_list']
    B_list = data['B_list']
    X = data['X']
    Y = data['Y']
    ef_T_marker = data['ef_T_marker']
    Base_T_viconW = data['Base_T_viconW']



In [ ]:
from SimpleHandEye.interfaces.ros2 import ROS2ExecutorManager, ROS2CameraReader, ROS2TFInterface
import rclpy
rclpy.init()    
executor_manager = ROS2ExecutorManager()
vicon_sensor = ROS2TFInterface('vicon/World', 'vicon/WHITE_FR3_EF/WHITE_FR3_EF', node_name='vicon_sensor_node')
executor_manager.add_node(vicon_sensor)
executor_manager.start()

In [ ]:
world_T_marker = vicon_sensor.get_pose()
world_T_marker

In [ ]:
 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

In [ ]:
from FR3Py.robot.model import PinocchioModel
model = PinocchioModel()

In [ ]:
import numpy as np
def getFK(robot, model):
    state = robot.getJointStates()
    q , dq = state['q'], state['dq']
    info = model.getInfo(q, dq)
    R_ee = info['R_EE']
    t_ee = info['P_EE']
    T = np.hstack([R_ee, t_ee.reshape(3,1)])
    T = np.vstack([T, np.array([0, 0, 0, 1])])
    return T

base_T_ef= getFK(robot, model)
base_T_ef

In [ ]:
base_T_world = base_T_ef @ ef_T_marker @ np.linalg.inv(world_T_marker)
T = base_T_world @ vicon_sensor.get_pose()

In [ ]:
with open('base_T_world.pkl', 'wb') as f:
    pickle.dump(base_T_world, f)

In [ ]:
from FR3ViconVisualizer.fr3_mj_env_collision_flying_ball import FR3MuJocoEnv
import numpy as np

In [ ]:
env = FR3MuJocoEnv()
env.reset()

In [ ]:
import time 
for i in range(100000):
    time.sleep(0.01)
    T = base_T_world @ vicon_sensor.get_pose()
    env.visualize_object(robot.getJointStates()['q'], T)